In [90]:
from V6_tratamiento import tratamiento
from utils import *

import pandas as pd
import numpy as np
from feature_engine.imputation import RandomSampleImputer
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import *

In [91]:
df_train = pd.read_pickle("./pickles_temp/V5_2_base.pkl") 
df_train

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,extraction_type_class,management_group,payment_type,quality_group,quantity,source_type,source_class,waterpoint_type,status_group,year_recorded
0,69572,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,XEgV,0,Lake Nyasa,...,gravity,user-group,annually,good,enough,spring,groundwater,communal standpipe,functional,2011
1,8776,0.0,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,gravity,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,functional,2013
2,34310,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,gravity,user-group,per bucket,good,enough,dam,surface,communal standpipe multiple,functional,2013
3,67743,0.0,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,submersible,user-group,never pay,good,dry,borehole,groundwater,communal standpipe multiple,non functional,2013
4,19728,0.0,Action In A,1527.0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,gravity,other/unknown,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,functional,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,Germany Republi,1210.0,CES,37.169807,-3.253847,Area Three Namba 27,0,Pangani,...,gravity,user-group,per bucket,good,enough,spring,groundwater,communal standpipe,functional,2013
59396,27263,4700.0,Cefa-njombe,1212.0,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,Rufiji,...,gravity,user-group,annually,good,enough,river/lake,surface,communal standpipe,functional,2011
59397,37057,0.0,9JgB,1532.5,9xav,34.017087,-8.750434,Mashine,0,Rufiji,...,handpump,user-group,monthly,fluoride,enough,borehole,groundwater,hand pump,functional,2011
59398,31282,0.0,Malec,820.5,Musa,35.861315,-6.378573,Mshoro,0,Rufiji,...,handpump,user-group,never pay,good,insufficient,shallow well,groundwater,hand pump,functional,2011


In [103]:
# 1- Voy a probar a dar label encoder a todo y ver si aumenta o disminuye desde el principio
df_train_1 = df_train.copy()
df_train_1['status_group'].replace(['non functional', 'functional needs repair', 'functional'], [0, 1, 2], inplace=True)


In [104]:
df_train_1.head(20)

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,extraction_type_class,management_group,payment_type,quality_group,quantity,source_type,source_class,waterpoint_type,status_group,year_recorded
0,69572,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,XEgV,0,Lake Nyasa,...,gravity,user-group,annually,good,enough,spring,groundwater,communal standpipe,2,2011
1,8776,0.0,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,gravity,user-group,never pay,good,insufficient,rainwater harvesting,surface,communal standpipe,2,2013
2,34310,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,gravity,user-group,per bucket,good,enough,dam,surface,communal standpipe multiple,2,2013
3,67743,0.0,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,submersible,user-group,never pay,good,dry,borehole,groundwater,communal standpipe multiple,0,2013
4,19728,0.0,Action In A,1527.0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,gravity,other/unknown,never pay,good,seasonal,rainwater harvesting,surface,communal standpipe,2,2011
5,9944,20.0,Mkinga Distric Coun,1411.5,DWE,39.172796,-4.765587,Tajiri,0,Pangani,...,submersible,user-group,per bucket,salty,enough,other,groundwater,communal standpipe multiple,2,2011
6,19816,0.0,Dwsp,1943.5,DWSP,33.362410,-3.766365,Kwa Ngomho,0,Internal,...,handpump,user-group,never pay,good,enough,borehole,groundwater,hand pump,0,2012
7,54551,0.0,Rwssp,1142.5,DWE,32.620617,-4.226198,Tushirikiane,0,Lake Tanganyika,...,handpump,user-group,other/unknown,milky,enough,shallow well,groundwater,hand pump,0,2012
8,53934,0.0,Wateraid,1722.5,Water Aid,32.711100,-5.146712,Kwa Ramadhan Musa,0,Lake Tanganyika,...,handpump,user-group,never pay,salty,seasonal,borehole,groundwater,hand pump,0,2012
9,46144,0.0,Isingiro Ho,1473.5,Artisan,30.626991,-1.257051,Kwapeto,0,Lake Victoria,...,handpump,user-group,never pay,good,enough,shallow well,groundwater,hand pump,2,2011


In [105]:
cat_cols= df_train_1.select_dtypes(include=['object','category']).columns
num_cols = df_train_1.select_dtypes(exclude=['object','category']).columns

In [106]:
le = LabelEncoder()
for variable in cat_cols:
    df_train_1[variable] = le.fit_transform(df_train_1[variable])
df_train_1

,id,amount_tsh,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,extraction_type_class,management_group,payment_type,quality_group,quantity,source_type,source_class,waterpoint_type,status_group,year_recorded
0,69572,6000.0,3376,1390.0,3477,34.938093,-9.856322,39031,0,1,...,0,3,0,2,1,6,0,1,2,2011
1,8776,0.0,1709,1399.0,1750,34.698766,-2.147466,39277,0,4,...,0,3,2,2,2,3,1,1,2,2013
2,34310,25.0,2413,686.0,4397,37.460664,-3.821329,15803,0,5,...,0,3,5,2,1,1,1,2,2,2013
3,67743,0.0,3962,263.0,4002,38.486161,-11.155298,39367,0,7,...,5,3,2,2,0,0,0,2,0,2013
4,19728,0.0,794,1527.0,906,31.130847,-1.825359,37222,0,4,...,0,1,2,2,3,3,1,1,2,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,1663,1210.0,1093,37.169807,-3.253847,1135,0,5,...,0,3,5,2,1,6,0,1,2,2013
59396,27263,4700.0,1118,1212.0,1184,35.249991,-9.070629,25305,0,6,...,0,3,0,2,1,4,1,1,2,2011
59397,37057,0.0,667,1532.5,711,34.017087,-8.750434,29244,0,6,...,1,3,1,1,1,0,0,4,2,2011
59398,31282,0.0,2535,820.5,2852,35.861315,-6.378573,31034,0,6,...,1,3,2,2,2,5,0,4,2,2011


In [107]:
model_setup = setup(data=df_train_1, 
                    target = 'status_group', 
                    )

,Description,Value
0,Session id,7227
1,Target,status_group
2,Target type,Multiclass
3,Original data shape,"(59400, 31)"
4,Transformed data shape,"(59400, 31)"
5,Transformed train set shape,"(41580, 31)"
6,Transformed test set shape,"(17820, 31)"
7,Numeric features,30
8,Preprocess,True
9,Imputation type,simple


In [109]:
if 0:
    best_model = compare_models()    # 4'
else:
    best_model = create_model('rf')  # 21s
# el mejor es el random forest
# da 0.8024. La V0 sin tocar nada dio 0.8061

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7992,0.9007,0.7992,0.7907,0.7883,0.6196,0.6251
1,0.8127,0.9051,0.8127,0.8074,0.8030,0.6453,0.6513
2,0.8066,0.9021,0.8066,0.8011,0.7976,0.6345,0.6401
3,0.8057,0.9100,0.8057,0.8012,0.7959,0.6315,0.6383
4,0.7949,0.8937,0.7949,0.7898,0.7850,0.6110,0.6176
5,0.7956,0.8946,0.7956,0.7888,0.7847,0.6121,0.6183
6,0.8167,0.9107,0.8167,0.8098,0.8070,0.6540,0.6587
7,0.8047,0.8994,0.8047,0.8004,0.7952,0.6300,0.6364
8,0.7946,0.9020,0.7946,0.7884,0.7826,0.6085,0.6172


In [110]:
best_model

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=100, n_jobs=-1,
                       oob_score=False, random_state=7227, verbose=0,
                       warm_start=False)

In [111]:
corr = abs(df_train_1.corr())
corr[['status_group']].sort_values(by = 'status_group',ascending = False).style.background_gradient()

,status_group
status_group,1.000000
waterpoint_type,0.206507
construction_year,0.184744
extraction_type_class,0.175338
quantity,0.172408
quality_group,0.153163
region_code,0.108640
gps_height,0.107153
region,0.091252
source_type,0.079656


In [112]:
# Lo subo a la web para comparar 
df_target = pd.read_csv('./test set values.csv', sep=',')
df_target = tratamiento(df_target)

In [116]:
def prediccion_final(df_target, best_model):
    cat_cols= df_target.select_dtypes(include=['object','category']).columns
    num_cols = df_target.select_dtypes(exclude=['object','category']).columns
    le = LabelEncoder()
    for variable in cat_cols:
        df_target[variable] = le.fit_transform(df_target[variable])

    df_predictions = predict_model(best_model, data=df_target)
    df_result = df_predictions[['id', 'prediction_label']]
    df_result.rename(columns={'prediction_label': 'status_group'}, inplace=True)
    df_result.set_index('id', inplace=True)
    return (df_result)

df_result = prediccion_final(df_target, best_model)
df_result['status_group'].replace([0, 1, 2], ['non functional', 'functional needs repair', 'functional'], inplace=True)
df_result.to_csv("./resultados/V6_0_tratamiento.csv")

# La web devuelve un 0.7863

       status_group
id                 
50785             0
51630             2
17168             2
45559             0
49871             2
...             ...
39307             0
18990             2
28749             2
33492             2
68707             0

[14850 rows x 1 columns]
         status_group
id                   
50785  non functional
51630      functional
17168      functional
45559  non functional
49871      functional
...               ...
39307  non functional
18990      functional
28749      functional
33492      functional
68707  non functional

[14850 rows x 1 columns]


In [ ]:
# hay que inventar columnas nuevas y probar a tirar distintos modelos

In [ ]:
#